In [1]:
import pandas as pd
import numpy as np
import re

# 여가 관련 자료

## 일과 여가 생활 균형

In [2]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')
# leisure.info()
# leisure.head(10)

In [4]:
leisure_T = leisure.T   # 행열 전환
# leisure_T.head(5)

In [5]:
leisure_T.replace("-",0,inplace=True)

In [6]:
leis_uq = leisure.iloc[:,0].unique()
leis_uq = leis_uq.tolist()   # ndarray -> list
leis_uq = [re.sub(r"\(+.\)","",uq.replace(" ","_")) for uq in leis_uq]  # 괄호와 괄호 안 글자, 공백 제거
print(leis_uq)

['통계분류', '전체', '성별', '연령별', '학력별', '동거가구원수별', '혼인상태별', '가구주여부별', '종사자지위별', '가구소득별', '지역규모별', '권역별', '17개_시도별', '장애여부별']


In [7]:
leisure_T.columns = leisure_T.iloc[0,:]
# leisure_T.info()

In [8]:
columns_nospace = [re.sub(r"\(+.\)", "", col.replace(" ","_")) for col in leisure_T.columns]

In [9]:
leisure_T.columns = columns_nospace
print(leisure_T.columns)

Index(['통계분류', '전체', '성별', '성별', '연령별', '연령별', '연령별', '연령별', '연령별', '연령별',
       '연령별', '학력별', '학력별', '학력별', '학력별', '동거가구원수별', '동거가구원수별', '동거가구원수별',
       '혼인상태별', '혼인상태별', '혼인상태별', '가구주여부별', '가구주여부별', '종사자지위별', '종사자지위별',
       '종사자지위별', '종사자지위별', '종사자지위별', '종사자지위별', '종사자지위별', '종사자지위별', '종사자지위별',
       '가구소득별', '가구소득별', '가구소득별', '가구소득별', '가구소득별', '가구소득별', '가구소득별', '지역규모별',
       '지역규모별', '지역규모별', '권역별', '권역별', '권역별', '권역별', '권역별', '권역별', '17개_시도별',
       '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별',
       '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별',
       '17개_시도별', '17개_시도별', '17개_시도별', '17개_시도별', '장애여부별', '장애여부별', '장애여부별'],
      dtype='object')


In [10]:
# 통계 분류별로 데이터 프레임 분류

for cat in leis_uq : 
    globals()["leisure_{}".format(cat)] = leisure_T[cat]
    
    if type(globals()["leisure_{}".format(cat)]) == pd.core.series.Series :   # type이 시리즈라면
        globals()["leisure_{}".format(cat)] = pd.DataFrame(globals()["leisure_{}".format(cat)])   # 시리즈를 데이터프레임으로 변경
        
    globals()["leisure_{}".format(cat)].columns = globals()["leisure_{}".format(cat)].iloc[1,:]
    globals()["leisure_{}".format(cat)] = globals()["leisure_{}".format(cat)].iloc[2:,:]
    globals()["leisure_{}".format(cat)].iloc[:,1:] = globals()["leisure_{}".format(cat)].iloc[:,1:].astype(float)

In [11]:
leis_uq2 = leis_uq[1:]
print(leis_uq2)

['전체', '성별', '연령별', '학력별', '동거가구원수별', '혼인상태별', '가구주여부별', '종사자지위별', '가구소득별', '지역규모별', '권역별', '17개_시도별', '장애여부별']


In [12]:
for uq2 in leis_uq2 : 
    globals()["leis_{0}_{1}".format(uq2, "합본")] = pd.concat([leisure_통계분류, globals()["leisure_{}".format(uq2)]],axis=1)

In [13]:
## 연도 리스트
year_List = [2016,2018,2019,2020,2021]

In [14]:
for uniq in leis_uq2 : 
    idx_List = globals()["leis_{0}_{1}".format(uniq, "합본")].index.to_list()   # 분리된 데이터프레임 인덱스 리스트로 반환
    
    for yr in year_List : 
        sep_year = []    # 정규표현식에 만족하는 인덱스를 저장할 리스트
        
        for idx in idx_List :
            find_yr = re.findall(str(yr)+"[.0-9]*",idx)     # 연도 + '.' or 숫자 1개 이상 포함돼 있는 인덱스 저장 

            if find_yr:      # 빈 리스트가 아니라면 (findall은 조건에 안 맞으면 빈 리스트 반환)
                sep_year.append(find_yr[0])

        Last_idx = len(sep_year)
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, "합본")][:][sep_year[0]:sep_year[Last_idx-1]]    # 행 인덱스로 슬라이싱, 행 인덱스로 슬라이싱할 때 콜론 뒤 값을 포함해서 슬라이싱한다.
        pattern = str(yr) + r'[.0-9]*'
        new_index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.map(lambda x: re.sub(pattern, str(yr), x))   # map 함수를 이용해 인덱스마다 정규 표현식 sub를 적용
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = new_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.astype(int)   # 인덱스를 int 타입으로
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"통계분류(2)" : "분류"}, inplace=True)   # 컬럼명 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].reset_index()   # reset_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"index" : "연도"}, inplace=True)    # 인덱스가 컬럼으로 바뀌고, 그 컬럼명을 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].set_index(["연도","분류"])   # 멀티 인덱스 설정


# 전체 -> 리스트 중에서 2016년이면 리스트에 저장 -> 리스트에 저장된 첫번째와 마지막 인덱스로 슬라이싱 -> 새로운 데이터 프레임 생성

In [ ]:
leis_17개_시도별_2016_T = leis_17개_시도별_2016.T
leis_17개_시도별_2016_T.head(10)

연도         2016                                                       \
분류          표본수 일에 더 집중하고있다 일에 조금 더 집중하고 있다 다소 일에 집중하고 있다 균형을 이루고 있다   
통계분류(2)                                                                
서울         1257         0.7             7.5          17.6       27.8   
부산        751.0         2.2             7.9          13.4       32.9   
대구        633.0         0.8             6.0          27.6       49.5   
인천        675.0         1.0             3.4          18.3       38.0   
광주        492.0         2.0             7.4          18.4       28.8   
대전        501.0         1.8            10.5          19.8       38.2   
울산        426.0         1.5            12.7          10.4       33.6   
세종        175.0         0.5             4.5          27.4       19.2   
경기       1357.0         0.8             7.8          23.3       40.9   
강원        462.0         6.1            10.4          24.0       39.4   

연도                                                      
분류      다소 여가에 더 집중하고있다 여가에 조금 더 집중하고 있다 여가에 더 집중하고 있다  
통계분류(2)                                                 
서울                 26.1             19.7           0.5  
부산                 29.3             12.8           1.6  
대구                  8.5              7.4           0.3  
인천                 27.6             11.2           0.5  
광주                 29.4             12.4           1.7  
대전                 22.9              6.4           0.3  
울산                 29.2             11.8           0.8  
세종                 24.8             23.5           0.0  
경기                 19.7              7.5           0.2  
강원                 13.5              5.3           1.3

In [132]:
df = leis_17개_시도별_2016_T.stack(level=0)
pd.DataFrame(df)


,분류,균형을 이루고 있다,다소 여가에 더 집중하고있다,다소 일에 집중하고 있다,여가에 더 집중하고 있다,여가에 조금 더 집중하고 있다,일에 더 집중하고있다,일에 조금 더 집중하고 있다,표본수
통계분류(2),연도,,,,,,,,
서울,2016,27.8,26.1,17.6,0.5,19.7,0.7,7.5,1257
부산,2016,32.9,29.3,13.4,1.6,12.8,2.2,7.9,751.0
대구,2016,49.5,8.5,27.6,0.3,7.4,0.8,6.0,633.0
인천,2016,38.0,27.6,18.3,0.5,11.2,1.0,3.4,675.0
광주,2016,28.8,29.4,18.4,1.7,12.4,2.0,7.4,492.0
대전,2016,38.2,22.9,19.8,0.3,6.4,1.8,10.5,501.0
울산,2016,33.6,29.2,10.4,0.8,11.8,1.5,12.7,426.0
세종,2016,19.2,24.8,27.4,0.0,23.5,0.5,4.5,175.0
경기,2016,40.9,19.7,23.3,0.2,7.5,0.8,7.8,1357.0


In [114]:
pd.concat([leis_17개_시도별_2016, leis_가구소득별_2016],axis=1)

통계분류(2)                  서울     부산     대구     인천     광주     대전     울산     세종  \
연도   분류                                                                        
2016 표본수               1257  751.0  633.0  675.0  492.0  501.0  426.0  175.0   
     일에 더 집중하고있다        0.7    2.2    0.8    1.0    2.0    1.8    1.5    0.5   
     일에 조금 더 집중하고 있다    7.5    7.9    6.0    3.4    7.4   10.5   12.7    4.5   
     다소 일에 집중하고 있다     17.6   13.4   27.6   18.3   18.4   19.8   10.4   27.4   
     균형을 이루고 있다        27.8   32.9   49.5   38.0   28.8   38.2   33.6   19.2   
     다소 여가에 더 집중하고있다   26.1   29.3    8.5   27.6   29.4   22.9   29.2   24.8   
     여가에 조금 더 집중하고 있다  19.7   12.8    7.4   11.2   12.4    6.4   11.8   23.5   
     여가에 더 집중하고 있다      0.5    1.6    0.3    0.5    1.7    0.3    0.8    0.0   

통계분류(2)                    경기     강원  ...     경북     경남     제주 100만원 미만  \
연도   분류                               ...                                 
2016 표본수               1357.0  462.0  ...  669.0  719.0  300.0      861   
     일에 더 집중하고있다          0.8    6.1  ...    2.2    1.4   10.8      0.9   
     일에 조금 더 집중하고 있다      7.8   10.4  ...   13.2    5.1   20.9      4.2   
     다소 일에 집중하고 있다       23.3   24.0  ...   24.3   14.8   18.8     12.2   
     균형을 이루고 있다          40.9   39.4  ...   35.5   37.1   29.6       31   
     다소 여가에 더 집중하고있다     19.7   13.5  ...    8.6   29.2    6.7     28.9   
     여가에 조금 더 집중하고 있다     7.5    5.3  ...   14.7   12.0   10.7       18   
     여가에 더 집중하고 있다        0.2    1.3  ...    1.5    0.3    2.7      4.8   

통계분류(2)               100~200만원 200~300만원 300~400만원 400~500만원 500~600만원  \
연도   분류                                                                   
2016 표본수                 1075.0    1594.0    2437.0    2231.0    1356.0   
     일에 더 집중하고있다            1.5       2.5       1.2       1.4       1.7   
     일에 조금 더 집중하고 있다        8.1      10.3       9.1       8.5       8.1   
     다소 일에 집중하고 있다         14.1      19.2      22.1      21.0      22.4   
     균형을 이루고 있다            35.3      32.4      36.5      36.3      37.1   
     다소 여가에 더 집중하고있다       24.7      23.6      19.6      22.0      20.0   
     여가에 조금 더 집중하고 있다      14.7      11.3      11.0      10.6      10.4   
     여가에 더 집중하고 있다          1.6       0.6       0.4       0.3       0.4   

통계분류(2)               600만원 이상  
연도   분류                         
2016 표본수                1048.0  
     일에 더 집중하고있다           2.1  
     일에 조금 더 집중하고 있다       8.6  
     다소 일에 집중하고 있다        21.6  
     균형을 이루고 있다           34.8  
     다소 여가에 더 집중하고있다      21.6  
     여가에 조금 더 집중하고 있다     10.6  
     여가에 더 집중하고 있다         0.8  

[8 rows x 24 columns]